In [1]:
import torch

Tensor: it just like numpy array but it few more features needed for neural network like it can work with GPU to exclarate process unlike numpy which can only work with CPU second is it can auto calculate gradient which to good to complex functions in NN

trailing _ in functions means our function will do inplace operations


vector can be 1d tensor  
matrix can be 2d tensor  
tensor can n-dimension array like numpy

In [ ]:
import torch

# Creating a 1D tensor
x = torch.tensor([1.0, 2.0, 3.0])
print('1D Tensor: \n', x)

# Creating a 2D tensor
y = torch.zeros((3, 3)) # same as np.zeros
print('2D Tensor: \n', y)

1D Tensor: 
 tensor([1., 2., 3.])
2D Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [33]:
a = torch.tensor([1.0, 2.0])
b = torch.tensor([3.0, 4.0])
print(a.view(2,1)) # view is same as resize in numpy used to resize the tensor but only diff is it dont change original tensor 
# Element-wise addition
print('Element Wise Addition of a & b: \n', a + b)

# Matrix multiplication
print('Matrix Multiplication of a & b (using torch.matmul   (matrix multiply)): \n', 
      torch.matmul(a.view(2,1), b.view(1,2)))    # @ works same as tensor.matmul
print('Matrix Multiplication of a & b (using @): \n', 
      a.view(2,1) @ b.view(1,2))    # @ works same as torch.matmul

tensor([[1.],
        [2.]])
Element Wise Addition of a & b: 
 tensor([4., 6.])
Matrix Multiplication of a & b (using torch.matmul   (matrix multiply)): 
 tensor([[3., 4.],
        [6., 8.]])
Matrix Multiplication of a & b (using @): 
 tensor([[3., 4.],
        [6., 8.]])


rehape gives diffrent arr with diffrent memory imean it is work like as deapcopy in fundamental python but it is slower than view  
view need continuos memory while reshpe does not need continuos memory

continuous memory means index of tenser is same as index of memory location of that element it affect in operations like transpose when actual location of element is not change but view is changed it give error doing further view oprations

.t a view opration works only on 2d array same as numpy 
.transform also a view operation but can works with any dimensions it swap between two dimensions (dim0, dim1)

In [36]:
import torch 
t = torch.tensor([[1, 2, 3, 4],
                 [5, 6, 7, 8],
                 [9, 10, 11, 12]])

# Reshaping
print("Reshaping")
print(t.reshape(6, 2))

# Resizing (deprecated, use reshape)
print("\nResizing")
print(t.view(2, 6))

# Transposing
print("\nTransposing")
print(t.transpose(0, 1))


x = torch.arange(9).reshape(3,3)
y = x.t()  # transpose, not contiguous
try:
    y.view(9)
except:
    print('error in view')
try:
    y.reshape(9)
except:
    print('error in reshape')

Reshaping
tensor([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12]])

Resizing
tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12]])

Transposing
tensor([[ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11],
        [ 4,  8, 12]])
error in view


## Autograd and Computational Graphs
The autograd module automates gradient calculation for backpropagation. This is crucia in training deep neural networks.

we can apply any algebra to tensor same as like in numpy(in parallel) we can also enable requires_grad so that our tensor will store all operation perform on it and calculate its gradient  
gradient could be calculated result.backward()

In [22]:
x = torch.tensor([2.0], requires_grad=True)
print(x)
y = x**2 + 3*x + 2 # y = x^2 +3x +2
y.backward()   # compute dy/dx = 2x + 3
print(x.grad)  # -> tensor([7.]) (2*2 + 3 = 7)
print(y)


tensor([2.], requires_grad=True)
tensor([7.])
tensor([12.], grad_fn=<AddBackward0>)


In [69]:
import numpy as np
x = torch.tensor(np.linspace(-5,5,500), requires_grad=True)
y1 = torch.tanh(x)
y3 = y1.mean()
y3.backward()   # compute dy/dx = 2x + 3
#print(x.grad)  # -> tensor([7.]) (2*2 + 3 = 7)


In [68]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_scatter(x=x.detach().numpy(),y=y1.detach().numpy())
fig.add_scatter(x=x.detach().numpy(),y=x.grad.detach().numpy())
fig.show()

simple neural network on minist dataset

In [129]:

import torch
import torch.nn as nn
import torch.nn.functional as F

xor_X = torch.tensor([[0,0],[0,1],[1,0],[1,1]],dtype=torch.float)
xor_Y = torch.tensor([0,1,1,0],dtype=torch.float).reshape(4,1)
class nn:
    def __init__(self, input, output,activation, *hidden_layer,lr):
        self.weights = []
        if hidden_layer:
            self.weights.append(torch.randn((input,hidden_layer[0]),requires_grad=True))
            for l in range(1,len(hidden_layer)):
                self.weights.append(torch.randn((hidden_layer[l-1],hidden_layer[l]),requires_grad=True))
            self.weights.append(torch.randn((hidden_layer[-1],output),requires_grad=True))
                
        else:
            self.weights.append(torch.randn((input, output),requires_grad=True))
        self.lr = lr
        self.act = activation
        #print(self.weights)
    def forward(self, input):
        for weight in self.weights:
            output = input @ weight
            input = self.act(output)
        return input
    def loss(self, y_true, y_pred):
        eps = 1e-9  # small constant to prevent log(0)
        return -(y_true * torch.log(y_pred + eps) + (1 - y_true) * torch.log(1 - y_pred + eps))

    def backpropagation(self, loss):
        loss.backward()
        with torch.no_grad():
            for i in range(len(self.weights)):
                self.weights[i] -= self.lr*self.weights[i].grad
                self.weights[i].grad.zero_()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = nn(2,1,torch.sigmoid,100,10,lr=0.01)
for epoch in range(10000):
    y_pred = model.forward(xor_X)
    loss = model.loss(xor_Y,y_pred).mean()
    model.backpropagation(loss)

    if epoch % 9999 == 0:
        print(f"epoch {epoch} loss {loss}")
        #print(model.weights)

y_pred = model.forward(xor_X)
print(y_pred)


epoch 0 loss 1.1599655151367188
epoch 9999 loss 0.060810185968875885
tensor([[0.0946],
        [0.9549],
        [0.9403],
        [0.0354]], grad_fn=<SigmoidBackward0>)


In [147]:
import random
def init_population(pop_size, max_layers=3, max_neurons=128):
    """Initialize population of networks with varying hidden layer sizes."""
    population = []
    for _ in range(pop_size):
        num_layers = random.randint(0, max_layers)   # how many hidden layers
        hidden_layers = [random.randint(1, max_neurons) for _ in range(num_layers)]
        population.append(hidden_layers)
    return population
def fitness_function(hidden_layers, accuracy, alpha=0.01, beta=0.001):
    """
    hidden_layers: list of hidden layer sizes
    accuracy: float (0 to 1)
    alpha: penalty weight for number of layers
    beta: penalty weight for total neurons
    """
    num_layers = len(hidden_layers)
    total_neurons = sum(hidden_layers) if hidden_layers else 0
    
    penalty = alpha * num_layers + beta * total_neurons
    fitness = accuracy - penalty *0  # maximize fitness
    return fitness
import random

def select_parents(population, accuracies, top_frac=0.4, bottom_frac=0.1):
    """
    population: list of architectures (list of hidden layer sizes)
    accuracies: list of accuracies for each individual
    Returns: selected parent architectures
    """
    # Compute fitness
    fitness_scores = [
        (fitness_function(population[i], accuracies[i]), population[i])
        for i in range(len(population))
    ]
    
    # Sort by fitness (descending, higher is better)
    fitness_scores.sort(key=lambda x: x[0], reverse=True)
    
    # Select top 40%
    n_top = max(1, int(len(population) * top_frac))
    best = [arch for _, arch in fitness_scores[:n_top]]
    
    # Select bottom 10%
    n_bottom = max(1, int(len(population) * bottom_frac))
    worst = [arch for _, arch in fitness_scores[-n_bottom:]]
    
    # Merge
    selected = best + worst
    return selected, fitness_scores

def crossover(parent1, parent2):
    """One-point crossover between two architectures (hidden layer lists)."""
    if not parent1:  # if parent1 empty
        return parent2.copy()
    if not parent2:  # if parent2 empty
        return parent1.copy()
    
    # Pick crossover points
    p1 = random.randint(0, len(parent1))
    p2 = random.randint(0, len(parent2))
    
    child = parent1[:p1] + parent2[p2:]
    return child

def mutate(individual, max_neurons=128, mutation_rate=0.3):
    """Mutate hidden layer architecture."""
    child = individual.copy()
    
    if random.random() < mutation_rate:
        action = random.choice(["add","remove","change"])
        
        if action == "add":
            new_layer = random.randint(1, max_neurons)
            pos = random.randint(0, len(child))
            child.insert(pos, new_layer)
            
        elif action == "remove" and child:
            pos = random.randrange(len(child))
            child.pop(pos)
            
        elif action == "change" and child:
            pos = random.randrange(len(child))
            child[pos] = random.randint(1, max_neurons)
            
    return child
def breed_population(parents, pop_size, mutation_rate=0.3, gen=1):
    """Create next generation from selected parents."""
    next_population = []
    while len(next_population) < pop_size:
        p1, p2 = random.sample(parents, 2)
        child = crossover(p1, p2)
        child = mutate(child, max_neurons=128*gen, mutation_rate=mutation_rate)
        next_population.append(child)
    return next_population



In [137]:
population = init_population(20, max_layers=20, max_neurons=100)
accuracy = []
best_loss = float('inf')
for pop_number, hiddenlayers in enumerate(population):
    model = nn(2,1,torch.sigmoid,*hiddenlayers,lr=0.01)
    for epoch in range(5000):
        y_pred = model.forward(xor_X)
        loss = model.loss(xor_Y,y_pred).mean()
        model.backpropagation(loss)
        if loss < best_loss:
            if epoch > 4998:
                print(f"population {pop_number} epoch {epoch} loss {loss}")
                #print(model.weights)
    accuracy.append(1-loss)     
    if loss < best_loss:
        print('hidden layer',hiddenlayers)
        print(f"population {pop_number}")
        y_pred = model.forward(xor_X)
        print(y_pred)
        best_loss = loss

population 0 epoch 4999 loss 0.23956814408302307
hidden layer [10, 78, 22, 98, 21, 44, 37, 17, 5]
population 0
tensor([[0.1376],
        [0.7257],
        [0.7750],
        [0.2083]], grad_fn=<SigmoidBackward0>)
population 1 epoch 4999 loss 0.07346685230731964
hidden layer [22, 70]
population 1
tensor([[0.0670],
        [0.9265],
        [0.9305],
        [0.0733]], grad_fn=<SigmoidBackward0>)
population 3 epoch 4999 loss 0.010990992188453674
hidden layer [37, 84, 95, 34, 56, 95, 71, 63, 92, 23, 49, 29, 53, 14, 51]
population 3
tensor([[0.0132],
        [0.9895],
        [0.9895],
        [0.0095]], grad_fn=<SigmoidBackward0>)
population 5 epoch 4999 loss 0.00828021951019764
hidden layer [69, 84, 14, 83, 23, 96, 13, 27, 27, 73, 37]
population 5
tensor([[0.0077],
        [0.9911],
        [0.9906],
        [0.0070]], grad_fn=<SigmoidBackward0>)
population 8 epoch 4999 loss 0.004878249950706959
hidden layer [57, 60, 48, 80, 76, 62, 64, 47]
population 8
tensor([[0.0046],
        [0.9964],

In [166]:
population = init_population(20, max_layers=20, max_neurons=128*1)
best_loss = float('inf')
genscore = []
for gen in range(1,20):    

    accuracy = []
    for pop_number, hiddenlayers in enumerate(population):
        model = nn(2,1,torch.sigmoid,*hiddenlayers,lr=0.01)
        for epoch in range(100):
            y_pred = model.forward(xor_X)
            loss = model.loss(xor_Y,y_pred).mean()
            model.backpropagation(loss)
            '''
            if loss < best_loss:
                if epoch :
                    print(f"population {pop_number} epoch {epoch} loss {loss}")
                    #print(model.weights)
            '''
        accuracy.append(1-loss)     
        if loss < best_loss:
            print('hidden layer',hiddenlayers)
            print(f"population {pop_number}")
            y_pred = model.forward(xor_X)
            print(y_pred)
            best_loss = loss
    parents, score = select_parents(population, accuracy)
    population = breed_population(parents, 15, gen)
    population.extend(init_population(5, max_layers=20, max_neurons=128))
    print('gen',gen, score[:3])
    genscore.append(zip(score,[gen]))

hidden layer [52, 6, 39, 73, 8, 55, 3, 127, 110, 53, 62, 40, 118, 111, 39, 72, 44]
population 0
tensor([[0.4998],
        [0.4998],
        [0.5003],
        [0.5000]], grad_fn=<SigmoidBackward0>)
hidden layer [92, 17, 88, 94, 105, 123, 46, 79, 112, 88, 5, 57]
population 4
tensor([[0.3918],
        [0.5022],
        [0.4981],
        [0.5064]], grad_fn=<SigmoidBackward0>)
hidden layer [117, 20, 40]
population 7
tensor([[0.4671],
        [0.6258],
        [0.4792],
        [0.4434]], grad_fn=<SigmoidBackward0>)
hidden layer [86, 50, 15, 64, 110, 120, 57]
population 16
tensor([[0.2675],
        [0.6471],
        [0.6606],
        [0.3133]], grad_fn=<SigmoidBackward0>)
gen 1 [(tensor(0.6133, grad_fn=<SubBackward0>), [86, 50, 15, 64, 110, 120, 57]), (tensor(0.3943, grad_fn=<SubBackward0>), [117, 20, 40]), (tensor(0.3768, grad_fn=<SubBackward0>), [53, 76, 46, 64, 40, 70, 96, 83, 96, 48, 42, 42, 56, 93, 36, 94])]
hidden layer [92, 68, 17, 88, 94, 105, 123, 46, 79, 112, 120, 57]
population 10

In [163]:
print(*zip(genscore))

(<zip object at 0x000001FAEE51F800>,) (<zip object at 0x000001FAEE51F1C0>,) (<zip object at 0x000001FAEE51EAC0>,) (<zip object at 0x000001FAEE51FB80>,) (<zip object at 0x000001FAEE550D40>,)


In [167]:
for i in genscore:
    print(*zip(i))

(((tensor(0.6133, grad_fn=<SubBackward0>), [86, 50, 15, 64, 110, 120, 57]), 1),)
(((tensor(0.9021, grad_fn=<SubBackward0>), [92, 68, 17, 88, 94, 105, 123, 46, 79, 112, 120, 57]), 2),)
(((tensor(0.8334, grad_fn=<SubBackward0>), [86, 50, 54, 64, 110, 120, 57]), 3),)
(((tensor(0.8377, grad_fn=<SubBackward0>), [125, 36, 53, 76, 46, 21, 117, 64, 70, 96, 83, 96, 48, 65, 57]), 4),)
(((tensor(0.9195, grad_fn=<SubBackward0>), [53, 76, 46, 40, 70, 96, 96, 48, 83, 96, 50, 65, 57, 73, 82]), 5),)
(((tensor(0.9640, grad_fn=<SubBackward0>), [53, 76, 46, 70, 96, 96, 48, 83, 96, 50, 65, 57, 88, 96, 125]), 6),)
(((tensor(0.9524, grad_fn=<SubBackward0>), [122, 28, 32, 71, 104, 59, 34, 116, 96, 48, 83, 115, 110, 120, 57]), 7),)
(((tensor(0.9297, grad_fn=<SubBackward0>), [122, 28, 32, 71, 104, 59, 34, 116, 96, 48, 83, 115, 110, 120, 57, 74, 76]), 8),)
(((tensor(0.9459, grad_fn=<SubBackward0>), [126, 59, 120, 91, 98, 126, 83, 120, 57]), 9),)
(((tensor(0.9179, grad_fn=<SubBackward0>), [76, 64, 110, 120, 87, 

In [103]:
import torch

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    # Get GPU name
    print("GPU:", torch.cuda.get_device_name(0))
    print("Current device index:", torch.cuda.current_device())

    # Create a tensor on GPU
    x = torch.rand((3,3), device="cuda")
    print("Tensor on GPU:\n", x)

    # Move it back to CPU
    y = x.to("cpu")
    print("Tensor moved back to CPU:\n", y)


CUDA available: True
GPU: NVIDIA GeForce RTX 2050
Current device index: 0


AcceleratorError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
